Before running this notebook you should have already run the crime.py script

In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.executor.memory", "5g")
    .config("spark.driver.memory", "5g")
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
24/09/24 12:18:49 WARN Utils: Your hostname, Mok resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/09/24 12:18:49 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/24 12:18:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [18]:
def save_df_to_parquet(name, sdf, folder):
    '''Input: take a spark dataframe, and a desired file name

    A staging folder is created as creating a parquet creates meta data files which are unneeded
    for analysis. Relevent parquet files are then moved into the curated folder
    
    Output: save the new spark data frame into the curated folder ready for analysis'''
    
    #now we want to move this file into the curated data
    import shutil
    import os
    
    folder_path = f'../data/staging'

    # Create the folder, including any intermediate directories
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created successfully!")
    else:
        None



    sdf.coalesce(1)\
        .write\
        .mode("overwrite")\
        .parquet(f'../data/staging/{name}')



    starting_directory = '../data/staging/'
    output_directory = f'../data/{folder}/'

    input_directory = os.path.join(starting_directory, name)

    for file in os.listdir(input_directory):
        if file.endswith(".parquet"):
            
            #rename the file first since currently they have a part-00000 name
            part_file_path = os.path.join(input_directory, file)
            new_file_path = os.path.join(input_directory, name)
            os.rename(part_file_path, new_file_path)
                
            #move the file into the curated folder for models
            source_file_path = os.path.join(input_directory, name)
            destination_file_path = os.path.join(output_directory, name)
        
            shutil.move(source_file_path, destination_file_path)
            shutil.rmtree(folder_path)

    print(f"Folder '{folder_path}' has been removed and file strucutre is cleaned!")

    return

In [19]:
crime_raw_sdf = spark.read.parquet('../data/landing/crime.parquet')

# we want to remove the subdivisions and LGAS as these are not relevent features 
offence_division_raw = crime_raw_sdf.drop("Offence Subdivision", "Offence Subgroup", "Year ending", "Local Government Area")

In [ ]:
offence_division_raw.show

In [20]:
#save this file as a parquet in the raw folder
save_df_to_parquet("offence_division_raw.parquet", offence_division_raw, "raw")

Folder '../data/staging' created successfully!


In [19]:
crime_raw_sdf = spark.read.parquet('../data/raw/offence_division_raw.parquet')

crime_raw_sdf.createOrReplaceTempView("offence_data")

# SQL query to group by 'Year', 'Suburb/Town Name', and 'Offence Division' and sum 'Offence Count'
query = """
    SELECT 
        Year,
        `Suburb/Town Name`,
        `Offence Division`,
        SUM(`Offence Count`) AS total_offence_count
    FROM offence_data
    GROUP BY Year, `Suburb/Town Name`, `Offence Division`
"""

# Execute the SQL query
result_df = spark.sql(query)

# Show the result
result_df.show()



+----+----------------+--------------------+-------------------+
|Year|Suburb/Town Name|    Offence Division|total_offence_count|
+----+----------------+--------------------+-------------------+
|2023|     Bakery Hill|E Justice procedu...|                  6|
|2023|    Lake Gardens|A Crimes against ...|                 12|
|2023| Trafalgar South|B Property and de...|                  3|
|2023|      Cheltenham|     C Drug offences|                 81|
|2023|  Echuca Village|A Crimes against ...|                  5|
|2023|           Patho|D Public order an...|                  4|
|2023|     Maryborough|A Crimes against ...|                221|
|2023|         Warrion|B Property and de...|                  2|
|2023|       Ellaswood|E Justice procedu...|                  3|
|2023|     Leitchville|D Public order an...|                  8|
|2023|      Lake Mundi|D Public order an...|                 11|
|2023|      Lethbridge|E Justice procedu...|                  8|
|2023|    Golden Gully|E 

In [20]:
from pyspark.sql.functions import when


sdf = result_df.withColumn(
    'Offence Division',
    when(result_df['Offence Division'] == 'A Crimes against the person', 1)
    .when(result_df['Offence Division'] == 'B Property and deception offences', 2)
    .when(result_df['Offence Division'] == 'C Drug offences', 3)
    .when(result_df['Offence Division'] == 'D Public order and security offences', 4)
    .when(result_df['Offence Division'] == 'E Justice procedures offences', 5)
    .when(result_df['Offence Division'] == 'F Other offences', 6)
    .otherwise(None)  # Use None or a default value if there is an unrecognized category
)

In [21]:
from itertools import product

years = [row['Year'] for row in sdf.select('Year').distinct().collect()]
suburbs = [row['Suburb/Town Name'] for row in sdf.select('Suburb/Town Name').distinct().collect()]

#Define all possible offence divisions
offence_divisions = [1, 2, 3, 4, 5, 6]  # Based on your specified divisions

#Create all combinations of Year, Suburb, and Offence Division
combinations = list(product(years, suburbs, offence_divisions))

# Create a DataFrame from these combinations
comb_df = spark.createDataFrame(combinations, schema=['Year', 'Suburb/Town Name', 'Offence Division'])

#Left join the original DataFrame with the combinations DataFrame
full_df = comb_df.join(sdf, on=['Year', 'Suburb/Town Name', 'Offence Division'], how='left')

#Fill missing offence counts with 0
full_df = full_df.fillna({'total_offence_count': 0})

full_df.show

<bound method DataFrame.show of +----+----------------+----------------+-------------------+
|Year|Suburb/Town Name|Offence Division|total_offence_count|
+----+----------------+----------------+-------------------+
|2014|        Ryanston|               1|                  0|
|2014|        Ryanston|               2|                  2|
|2014|        Ryanston|               3|                  0|
|2014|        Ryanston|               4|                  4|
|2014|        Ryanston|               5|                  0|
|2014|        Ryanston|               6|                  0|
|2014|    Neerim South|               1|                  8|
|2014|    Neerim South|               2|                 27|
|2014|    Neerim South|               3|                  7|
|2014|    Neerim South|               4|                  6|
|2014|    Neerim South|               5|                  2|
|2014|    Neerim South|               6|                  5|
|2014|          Thoona|               1|             

In [22]:
full_df.createOrReplaceTempView("offence_data")

query = """
    SELECT 
        Year,
        `Suburb/Town Name`,
        `Offence Division`,
        `total_offence_count`
    FROM offence_data
    GROUP BY Year, `Suburb/Town Name`, `Offence Division`, `total_offence_count`

"""

# Execute the SQL query
check_df = spark.sql(query)

# Show the result
check_df.show()

+----+-------------------+----------------+-------------------+
|Year|   Suburb/Town Name|Offence Division|total_offence_count|
+----+-------------------+----------------+-------------------+
|2014|       Tonghi Creek|               5|                  0|
|2014|         Callawadda|               1|                  2|
|2014|         Chum Creek|               6|                  0|
|2014|            Croydon|               3|                106|
|2014|     Dockers Plains|               5|                  0|
|2014|       Beech Forest|               4|                  0|
|2014|           Toolondo|               4|                  0|
|2014|          Newington|               6|                  0|
|2014|       Sunset Strip|               2|                  6|
|2014|          Benjeroop|               4|                  0|
|2014|        Albert Park|               1|                 45|
|2014|          East Sale|               6|                  0|
|2014|Woodstock On Loddon|              

In [23]:
from pyspark.sql import Window
from pyspark.sql.functions import lag, col

#Create a window specification that partitions by 'Suburb/Town Name' and 'Offence Division' and orders by 'Year'
window_spec = Window.partitionBy('Suburb/Town Name', 'Offence Division').orderBy('Year')

#Use the lag function to get the previous year's total offence count
df_with_prev_year = check_df.withColumn(
    'prev_year_offence_count',
    lag('total_offence_count').over(window_spec)
)

#Calculate the percentage change
df_with_percentage_change = df_with_prev_year.withColumn(
    'difference_change',
    ((col('total_offence_count') - col('prev_year_offence_count')))
)

#Show the result
df_with_percentage_change.show()


+----+----------------+----------------+-------------------+-----------------------+-----------------+
|Year|Suburb/Town Name|Offence Division|total_offence_count|prev_year_offence_count|difference_change|
+----+----------------+----------------+-------------------+-----------------------+-----------------+
|2014|        Abbeyard|               1|                  0|                   NULL|             NULL|
|2015|        Abbeyard|               1|                  0|                      0|                0|
|2016|        Abbeyard|               1|                  0|                      0|                0|
|2017|        Abbeyard|               1|                  0|                      0|                0|
|2018|        Abbeyard|               1|                  2|                      0|                2|
|2019|        Abbeyard|               1|                  0|                      2|               -2|
|2020|        Abbeyard|               1|                  0|             

In [24]:
from pyspark.sql.functions import concat, lit, to_date

# Assuming 'Year' is an integer column, convert it to a DateType with the first day of the year
df_schema_change = df_with_percentage_change

df_schema_change = df_schema_change.withColumnRenamed("Year", "year") \
       .withColumnRenamed("Suburb/Town Name", "suburb")\
       .withColumnRenamed("Offence Division", "offence_division")

df_schema_change = df_schema_change.withColumn("offence_division", col("offence_division").cast("integer")) \
       .withColumn("total_offence_count", col("total_offence_count").cast("integer"))\
       .withColumn("prev_year_offence_count", col("prev_year_offence_count").cast("integer"))\
       .withColumn("difference_change", col("difference_change").cast("integer"))\
       .withColumn("year", col("year").cast("integer"))

df_schema_change.schema

StructType([StructField('year', IntegerType(), True), StructField('suburb', StringType(), True), StructField('offence_division', IntegerType(), True), StructField('total_offence_count', IntegerType(), False), StructField('prev_year_offence_count', IntegerType(), True), StructField('difference_change', IntegerType(), True)])

In [25]:
df_with_percentage_changed = df_schema_change.select("suburb").distinct()
print(df_with_percentage_changed.count())

2849


In [26]:
from pyspark.sql.functions import lower
df_schema_change = df_schema_change.withColumn("suburb", lower(df_schema_change["suburb"]))

df_schema_change.show()

+----+--------+----------------+-------------------+-----------------------+-----------------+
|year|  suburb|offence_division|total_offence_count|prev_year_offence_count|difference_change|
+----+--------+----------------+-------------------+-----------------------+-----------------+
|2014|abbeyard|               1|                  0|                   NULL|             NULL|
|2015|abbeyard|               1|                  0|                      0|                0|
|2016|abbeyard|               1|                  0|                      0|                0|
|2017|abbeyard|               1|                  0|                      0|                0|
|2018|abbeyard|               1|                  2|                      0|                2|
|2019|abbeyard|               1|                  0|                      2|               -2|
|2020|abbeyard|               1|                  0|                      0|                0|
|2021|abbeyard|               1|                  

In [27]:

save_df_to_parquet("crimes.parquet", df_schema_change, "curated")
     

Folder '../data/staging' created successfully!
Folder '../data/staging' has been removed and file strucutre is cleaned!


In [30]:
import pandas as pd
import os
crime_df = pd.read_parquet('../data/curated/crimes.parquet')
crime_df.to_csv('../data/curated/crimes.csv')

os.remove('../data/curated/crimes.parquet')
print("removed the parquet version of the crimes file!")
  

removed the parquet version of the crimes file!
